# Time series Analysis - Air Quality #

##### Location - Some city in Italy #####
##### Time Period -  March 2004 to February 2005 #####

More details about the dataset can be found here - [link](https://archive.ics.uci.edu/ml/datasets/Air+Quality)

***

**Objective:**

Perform analysis on single features to know at their distribution. Also see relation between features. Can we make predictions

Before performing analysis on any dataset it is always a good idea to first list what your intuition about the analysis is. It may be incorrect, doesn't matter. The aim of this step is to avoid exploring the whole dataset without any direction.

This process is also called as *Hypothesis Generation*.

**Hypotheses**

1. Over time, the temperature must have increasing trend, excluding the seasonal variations
2. 